In [76]:
import json
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
data = []
url = "https://en.wikipedia.org/wiki/Costa_Rica_national_football_team"
page = requests.get(url)
soup = BeautifulSoup(page.text, parser='lxml')

#table = soup.select('table', attrs={"class":"sortable wikitable plainrowheaders jquery-tablesorter"})
table = soup.find( "table", {"class":"plainrowheaders"} )
table_body = table.find('tbody')

rows = table_body.find_all('tr')
for row in rows:
    cols = row.find_all(['td','th'])
    
    cols = [ele.text.strip() for ele in cols]
    
    data.append([ele for ele in cols]) # Get rid of empty values
    
tab1 = pd.DataFrame(data)
tab1.columns = tab1.iloc[0]
tab1 = tab1[1:]
tab1

,No.,Pos.,Player,Date of birth (age),Caps,Goals,Club
1,1,1GK,Keylor Navas,(1986-12-15) 15 December 1986 (age 35),108,0,Paris Saint-Germain
2,18,1GK,Esteban Alvarado,(1989-04-28) 28 April 1989 (age 33),25,0,Herediano
3,23,1GK,Patrick Sequeira,(1999-03-01) 1 March 1999 (age 23),2,0,Lugo
4,,None,None,None,None,None,None
5,3,2DF,Juan Pablo Vargas,(1995-06-06) 6 June 1995 (age 27),12,1,Millonarios
6,4,2DF,Keysher Fuller,(1994-07-12) 12 July 1994 (age 28),32,2,Herediano
7,6,2DF,Óscar Duarte,(1989-06-03) 3 June 1989 (age 33),72,4,Al-Wehda
8,8,2DF,Bryan Oviedo,(1990-02-18) 18 February 1990 (age 32),77,2,Real Salt Lake
9,15,2DF,Francisco Calvo,(1992-07-08) 8 July 1992 (age 30),76,8,Konyaspor
10,16,2DF,Carlos Martínez,(1999-03-30) 30 March 1999 (age 23),8,0,San Carlos


In [13]:
import json
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
data = []
url = "https://en.wikipedia.org/wiki/Costa_Rica_national_football_team"
page = requests.get(url)
soup = BeautifulSoup(page.text, parser='lxml')

#table = soup.select('table', attrs={"class":"sortable wikitable plainrowheaders jquery-tablesorter"})
table = soup.find( "table", {"class":"infobox"} )
table_body = table.find('tbody')

rows = table_body.find_all('tr')
for row in rows:
    cols = row.find_all(['td','th'])
    
    cols = [ele.text.strip() for ele in cols]
    
    data.append([ele for ele in cols]) # Get rid of empty values
    
tab1 = pd.DataFrame(data)
tab1.columns = tab1.iloc[0]
tab1 = tab1[1:]
tab1

AttributeError: 'NoneType' object has no attribute 'find_all'

In [17]:
import json
import requests
from bs4 import BeautifulSoup
def download_page(url):
    response = requests.get(url)
    response.raise_for_status()
    return response.text
def main(url):
    content = download_page(url)
    soup = BeautifulSoup(content, 'html.parser')
    result = {}
    for row in soup.table.find_all('thead'):
        row_header = row.th.get_text()
        row_cell = row.td.get_text()
        result[row_header] = row_cell
    with open('myCSV/book_table.json', 'w') as storage_file:
        storage_file.write(json.dumps(result))
if __name__ == "__main__":
    main("https://en.wikipedia.org/wiki/Costa_Rica_national_football_team")

In [1]:
import pandas as pd
import numpy as np
table = pd.read_html("https://en.wikipedia.org/wiki/Costa_Rica_national_football_team")        

table        
cols = {0:"ind",1:"val"}
columns = list(cols.values())
tab = pd.DataFrame(table[0])
tab
tab = tab.rename(columns=cols)
tab = tab.loc[:,columns]
tab = tab.dropna()
datas = ["Head coach","FIFA code","Confederation"]
dat = tab.loc[(tab["ind"].isin(datas))]
res = dat.set_index("ind").T
res = res.to_dict(orient="records")
res[0]

ValueError: invalid literal for int() with base 10: '2"'

In [4]:
import pandas as pd
import numpy as np
table = pd.read_html("https://en.wikipedia.org/wiki/Costa_Rica_national_football_team")        

cols = {0:"ind",1:"val"}
columns = list(cols.values())
table[0]

ValueError: invalid literal for int() with base 10: '2"'

In [8]:
import pandas as pd
import numpy as np
from app.functions import extractDataset, Teams
team = []
teamlist = extractDataset('FifaWC2022/linkdata.csv')
numdata = len(teamlist)
for i in range(numdata):
    t = Teams(i+1,teamlist.index[i],teamlist.iloc[i]["link"])
    print(t.teamid, " - ", t.teamname, " - ", t.link)
    team.append(t)
    
for i in range(len(team)):
    
    print(team[i].teamname)
    print(team[i].getTeamsHead())
    #print(team[1].getTeamsHead())
#print(team[0].getTeamsDetails('No. Pos.'))
#print(team[1].getTeamsDetails('No. Pos.'))

1  -  Australia  -  https://en.wikipedia.org/wiki/Australia_men%27s_national_soccer_team
2  -  Iran  -  https://en.wikipedia.org/wiki/Iran_national_football_team
3  -  Japan  -  https://en.wikipedia.org/wiki/Japan_national_football_team
4  -  Qatar  -  https://en.wikipedia.org/wiki/Qatar_national_football_team
5  -  Saudi Arabia  -  https://en.wikipedia.org/wiki/Saudi_Arabia_national_football_team
6  -  South Korea  -  https://en.wikipedia.org/wiki/South_Korea_national_football_team
7  -  Cameroon  -  https://en.wikipedia.org/wiki/Cameroon_national_football_team
8  -  Ghana  -  https://en.wikipedia.org/wiki/Ghana_national_football_team
9  -  Morocco  -  https://en.wikipedia.org/wiki/Morocco_national_football_team
10  -  Senegal  -  https://en.wikipedia.org/wiki/Senegal_national_football_team
11  -  Tunisia  -  https://en.wikipedia.org/wiki/Tunisia_national_football_team
12  -  Canada  -  https://en.wikipedia.org/wiki/Canada_men%27s_national_soccer_team
13  -  Costa Rica  -  https://en.

In [15]:
import pandas as pd
import numpy as np

table = pd.read_html("https://en.wikipedia.org/wiki/Ghana_national_football_team")
print(f'Total tables: {len(table)}')
table[3].info()

Total tables: 97
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Kit supplier  4 non-null      object
 1   Period        4 non-null      object
dtypes: object(2)
memory usage: 192.0+ bytes


In [13]:
import pandas as pd
import numpy as np
import requests

url = 'https://en.wikipedia.org/wiki/Costa_Rica_national_football_team'
html = requests.get(url).content

df_list = pd.read_html(html)
#pd.Series(['260,327,021']).str.replace(',', '').astype(int)
#df_list
#soup = BeautifulSoup(html)
#table = soup.select_one('table:contains("No. Pos.                 Player Date of birth (age)  Caps  Goals")')


ValueError: invalid literal for int() with base 10: '2"'

In [ ]:
soup = BeautifulSoup("https://en.wikipedia.org/wiki/Australia_men%27s_national_soccer_team")
table = soup.select_one('table:contains("Current squad")')
table
#table1 = pd.read_html("https://en.wikipedia.org/wiki/Ghana_national_football_team", match='No.')
table1 = pd.read_html("https://en.wikipedia.org/wiki/Australia_men%27s_national_soccer_team", match='No.')

print(f'Total tables: {len(table1)}')
ColumnNames = {"No.":"Number","Pos.":"Position","Player":"Player name","Date of birth (age)":"Date of birth",
               "Club":"Current club"}
tab = pd.DataFrame(table1[6])
tab = tab.dropna()
tab = tab.rename(columns=ColumnNames)
dtypescols = {"Number":"int","Caps":"int","Goals":"int"}
tab = tab.astype(dtypescols)
tab.index = np.arange(1, len(tab) + 1)
tab.index.name = "test"
tab

In [103]:
table = pd.read_html("https://en.wikipedia.org/wiki/Ghana_national_football_team")
print(f'Total tables: {len(table)}')
#cols = {0:"a",1:"b"}
tab = pd.DataFrame(table[8])
tab
#tab = tab.rename(columns=cols)
#tab = tab.loc[(tab["a"]=="Head coach")]["b"]
#tab.values[0]


Total tables: 97


,0,1,2,3,4
0,25 March 2022 FIFA World Cup qualification 1st...,Ghana,0–0,Nigeria,"Kumasi, Ghana"
1,19:30 UTC±0,NaN,Report,NaN,Stadium: Baba Yara StadiumReferee: Redouane Ji...


In [113]:
table = pd.read_html("https://en.wikipedia.org/wiki/Ghana_national_football_team")
print(f'Total tables: {len(table)}')
cols = {0:"ind",1:"val"}
tab = pd.DataFrame(table[0])
tab = tab.rename(columns=cols)
datas = ["Head coach","FIFA code","Confederation"]
dat = tab.loc[(tab["ind"].isin(datas))]
#dat = tab.loc[(tab["a"]=="Head coach") | (tab["a"]=="Confederation")]
#res = pd.DataFrame(dat["a"],dat["b"])
#dict(dat)
res = dat.set_index("ind").T
#res = dat.to_dict(orient="records")
res = res.to_dict(orient="records")
res

Total tables: 97


[{'Confederation': 'CAF (Africa)',
  'Head coach': 'Otto Addo',
  'FIFA code': 'GHA'}]

In [105]:
table = pd.read_html("https://en.wikipedia.org/wiki/Ghana_national_football_team")
print(f'Total tables: {len(table)}')
cols = {0:"a",1:"b"}
tab = pd.DataFrame(table[0])
tab

Total tables: 97


,0,1
0,NaN,NaN
1,Nickname(s),Black Stars
2,Association,Ghana Football Association (GFA)
3,Confederation,CAF (Africa)
4,Sub-confederation,WAFU (West Africa)
5,Head coach,Otto Addo
6,Captain,André Ayew
7,Most caps,André Ayew (110)
8,Top scorer,Asamoah Gyan (51)
9,Home stadium,Various
